## 1. Args and Import.

In [1]:
import time
import argparse
import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim
from models import GCN
from sklearn.metrics import f1_score
from utils import accuracy, set_seed

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
parser.add_argument('--epochs',type=int,default=200,help='Number of epochs to train')
parser.add_argument('--seed',type=int,default=42,help='random seed')
parser.add_argument('--lr',type=float,default=0.01,help='Initial learning rate,adam:0.001,sgd:0.05')
parser.add_argument('--weight_decay',type=float,default=5e-4,help='weight decay') #5e-4
parser.add_argument('--hidden',type=int,default=16,help='Number of hidden units')
parser.add_argument('--dropout',type=float,default=0.5,help='Dropout rate')
parser.add_argument('--dataset',type=str,default='Cora',help='name of dataset:Cora')
parser.add_argument('--optimize_type',type=str,default='Adam', help='Optimization method, Adam; SGD')
parser.add_argument('--model_name',type=str,default='GCN',help='name of model:GCN')
parser.add_argument('--norm_type',type=str,default='SymNorm_tildeA',help= 'Normalization method:SymNorm_tildeA')
parser.add_argument('--layer_num',type=int,default=6,help='layer num for GCN_nlayer')
parser.add_argument('--data_path', type=str,default='../data/', help='the path for dataset')
args = parser.parse_args(args=[])


ModuleNotFoundError: No module named 'torch_geometric'

In [2]:
print(args)

NameError: name 'args' is not defined

## 2. Load Data.

In [3]:
from dataloader import load_citation, load_citation_v2, load_citation_v3
import time
start_time = time.time()
dataf = args.data_path
norm_type = args.norm_type
#norm_type = "sym_normalized_A"
original_graph, L, features, labels, idx_train,idx_val, idx_test, data_package = load_citation_v3(dataf,
                                                                                                  args.dataset,
                                                                                                  norm_type=norm_type,
                                                                                                  cuda=True, 
                                                                                                  identity_features=False)


ModuleNotFoundError: No module named 'networkx'

In [ ]:
num_node = features.shape[0]
print('num_node:',features.shape[0])
num_feature = features.shape[1]  
print('num_feature: ',features.shape[1])
num_class = labels.max().item()+1
print('num_class',num_class)

In [ ]:
split_train = [i for i in range(num_node)]
idx_train = torch.LongTensor(split_train).cuda()
idx_test = torch.LongTensor(split_train).cuda()
idx_valid = torch.LongTensor(split_train).cuda()
print(idx_train.shape, idx_test.shape, idx_valid.shape)

## 3. Model and Optimizer Initialization.

In [ ]:
# 网络加深，反而效果差了
print(num_feature, args.hidden, num_class, args.dropout, args.layer_num, True)
if args.model_name == 'GCN':
    model = GCN( num_feature,args.hidden,num_class,args.dropout,bias=True).cuda()
else:
    raise Exception('Invalid model:',args.model_name)
if args.optimize_type == "Adam":
    optimizer = optim.Adam(model.parameters(),lr=args.lr,weight_decay=args.weight_decay)
else:
    optimizer = optim.SGD(model.parameters(),lr=args.lr,weight_decay=args.weight_decay)
print('model:%s, Norm:%s, optimizer: %s, hidden:%d'% (args.model_name, args.norm_type,args.optimize_type,args.hidden))
print(model)
def count_parameters(model):
    total_param = 0
    for name, param in model.named_parameters():
        if param.requires_grad:
            num_param = np.prod(param.size())
            if param.dim() > 1:
                print(name, ':', 'x'.join(str(x) for x in list(param.size())), '=', num_param)
            else:
                print(name, ':', num_param)
            total_param += num_param
    return total_param
print(count_parameters(model))

In [ ]:
def test(model, idx_test):
    model.eval()
    output = model(features,L)
    loss_test = F.nll_loss(output[idx_test],labels[idx_test])
    acc_test = accuracy(output[idx_test],labels[idx_test])
    print(loss_test.item(), acc_test.item())

def train(model, idx_train, idx_val):

    start = time.time()
    for epoch in range(args.epochs):
        t = time.time()
        model.train()
        optimizer.zero_grad()
        output = model(features,L)
        loss_train = F.nll_loss(output[idx_train],labels[idx_train])
        acc_train = accuracy(output[idx_train],labels[idx_train])
        loss_train.backward()
        optimizer.step()
        model.eval()
        output = model(features,L)
        loss_val = F.nll_loss(output[idx_val],labels[idx_val])
        acc_val = accuracy(output[idx_val],labels[idx_val])
        #label = labels[idx_val].cpu()
        #output = output[idx_val].max(1)[1].type_as(labels).cpu()
        #f1_val  = f1_score(label.numpy(),output.numpy(), average="micro")
        take_time = time.time()-t
        print("Epoch : {}, Time : {}, Val_loss: {}, Val_Acc: {}".format(epoch, take_time, loss_val.item(), acc_val.item()))
    

print("start training----")
for i in range(1):
    print(i)
    train(model, idx_train, idx_valid)
    test(model, idx_test)
print('finish------------')



In [ ]:
## Training GCN with all data in Cora.
Epoch : 199, Time : 0.010625362396240234, Val_loss: 0.129765585064888, Val_Acc: 0.9623338257016248


## 4. Save and Load Model.

In [ ]:
model_path = "gcn_identity_features_photo.pkt"
torch.save(model, model_path)
model = torch.load(model_path)
test()

In [ ]:
model_path = "gcn_identity_features_photo_state.pkt"
torch.save(model.state_dict(), model_path)

In [ ]:
model_path = "gcn_photo.pkt"
model = torch.load(model_path)
test()

## 5. Hook Model Definition.

In [ ]:
import math
import torch
import torch.nn as nn 
from torch.nn.parameter import Parameter
import torch.nn.functional as F
from torch.nn.modules.module import Module
from torch.autograd import Variable
import numpy as np
import skcuda.linalg as sklin
from layer import GCN_layer, BiGCN1_layer, BiGCN2_layer, BiGCN3_layer 

'''
GCN_layer(ind,outd,bias=True)
BiGCN1_layer(ind,outd,p,bias=True,beta=True,A2='cos_A2',n_iter=2)  
BiGCN2_layer(ind,outd,bias=True, beta=True, A2=None)
BiGCN3_layer(ind,outd,bias=True, beta=True, A2=None)
--A2 cos_A2/learn_A2/None
'''

class GCN_hook(nn.Module):
    def __init__(self, num_feature,num_hidden,num_class,dropout,bias=True):
        super(GCN_hook,self).__init__()

        self.gc1 = GCN_layer(num_feature, num_hidden)
        self.gc2 = GCN_layer(num_hidden, num_class)
        self.dropout = dropout

    def forward(self, x, adj):
        x = F.relu(self.gc1(x, adj))
        x1 = F.dropout(x, self.dropout, training=self.training)
        x2 = self.gc2(x1, adj)
        return F.log_softmax(x2, dim=1), x1

class GCN_3layer_hook(nn.Module):
    def __init__(self, num_feature,num_hidden,num_class,dropout,bias=True):
        super(GCN_3layer_hook,self).__init__()

        self.gc1 = GCN_layer(num_feature, num_hidden)
        self.gc2 = GCN_layer(num_hidden, num_hidden)
        self.gc3 = GCN_layer(num_hidden, num_class)
        self.dropout = dropout

    def forward(self, x, adj):
        x = F.relu(self.gc1(x, adj))
        x1 = F.dropout(x, self.dropout, training=self.training)
        x2 = F.relu(self.gc2(x1, adj))
        x2 = F.dropout(x2, self.dropout, training=self.training)
        x3 = self.gc3(x2, adj)
        return F.log_softmax(x3, dim=1), x1,x2
    
class GCN_nlayer_hook(nn.Module):
    def __init__(self, num_feature,num_hidden,num_class,dropout,layer_num=2,bias=True):
        super(GCN_nlayer_hook,self).__init__()
        self.layer_num = layer_num
        self.gc_input = GCN_layer(num_feature, num_hidden)
        if layer_num > 2:
            self.gc_inner = nn.ModuleList([GCN_layer(num_hidden, num_hidden) for i in range(layer_num-2)])        
        self.gc_output = GCN_layer(num_hidden, num_class)
        self.dropout = dropout

    def forward(self, x, adj):
        inner_state = []
        x = F.relu(self.gc_input(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        inner_state.append(x)
        if self.layer_num > 2:
            for i in range(self.layer_num - 2):
                x = self.gc_inner[i](x,adj)
                x = F.dropout(x, self.dropout, training=self.training)
                inner_state.append(x)
        x = self.gc_output(x, adj)
        return F.log_softmax(x, dim=1), inner_state

In [ ]:
model_path = "gcn_cora_%dlayer_symnorm_state.pkt".format(6)
#torch.save(model.state_dict(), model_path)
args_input = [1433,16,7,0.5]
kwargs = {
    "layer_num": 6,
    "bias": True
}
model = GCN_nlayer_hook(*args_input,**kwargs).cuda()
model.load_state_dict(torch.load(model_path))

model.eval()
output, inner_state = model(features,L)
loss_test = F.nll_loss(output[idx_test],labels[idx_test])
acc_test = accuracy(output[idx_test],labels[idx_test])
print(loss_test.item(), acc_test.item())


In [ ]:
model_path = "gcn_identity_features_cora.pkt"
model = torch.load(model_path)
model.eval()
output = model(features,L)
loss_test = F.nll_loss(output[idx_test],labels[idx_test])
acc_test = accuracy(output[idx_test],labels[idx_test])
print(loss_test.item(), acc_test.item())


## 6. Hidden State Visualization

In [ ]:
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import time
def dimension_reduction(input_array):
    start_time = time.time()
    X = np.array(input_array)    
    X_embedded = TSNE(n_components=2).fit_transform(X)
    print(time.time() - start_time)
    return X_embedded
def visualize(embedded_array,labels):
    
    #sns.set_palette(sns.color_palette("Paired"))
    X = np.array(embedded_array)
    labels = np.array(labels)
    labels = np.expand_dims(labels, axis=1)
    data = np.concatenate((X, labels), axis=1)
    df = pd.DataFrame(data, columns=["x", "y","Labels"])
    # Create an array with the colors you want to use
    colors = ["#FF0B04", "#4374B3"]
    # Set your custom color palette
    customPalette = sns.set_palette(sns.color_palette(colors))
    ax = sns.scatterplot(x="x", y="y",hue="Labels", data=df, palette="Set1", legend=False)
    
current_palette = sns.color_palette()
sns.palplot(current_palette)



In [ ]:
## Output
input_array = output.cpu().detach().numpy()
labels_array = labels.cpu().detach().numpy()
embedded_array = dimension_reduction(input_array)


In [ ]:
## Input
features_array = features.cpu().detach().numpy()
features_embeddded_array = dimension_reduction(features_array)

In [ ]:
## Layer2
inner_embedded_array = []
labels_array = labels.cpu().detach().numpy()

for i in range(len(inner_state)):
    layer2 = inner_state[i]
    layer2_array = layer2.cpu().detach().numpy()
    layer2_embeddded_array = dimension_reduction(layer2_array)
    inner_embedded_array.append(layer2_embeddded_array)


In [ ]:
## Layer3
layer3_array = layer3.cpu().detach().numpy()
layer3_embeddded_array = dimension_reduction(layer3_array)
labels_array = labels.cpu().detach().numpy()

In [ ]:
visualize(embedded_array, labels.cpu().detach().numpy())

In [ ]:
visualize(inner_embedded_array[3], labels.cpu().detach().numpy())

In [ ]:
print(layer2_embeddded_array)
import pickle as pkl
with open("cora_tsne_hidden.pkt","wb") as f:
    pkl.dump(layer2_embeddded_array, f)